# 1. Intro

Dataset Link: https://www.propublica.org/datastore/dataset/compas-recidivism-risk-score-data-and-analysis

Class slides with basic info about dataset:
https://docs.google.com/presentation/d/1RQZZpXmt1i-DyEEAZTFiBvrpuePFMJh69tXEWDPJIO8/edit#slide=id.g11f80bb4f73_0_115

Dataset Github: https://github.com/propublica/compas-analysis

A2 model github: https://github.com/mbilalzafar/fair-classification

# 2. Preparation

        2.1 Packages

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from scipy.optimize import fmin_slsqp
import sys
sys.path.insert(0, "/Users/xu/Desktop/5243 project4")
import utils as ut
import loss_funcs as lf

        2.2 Data Cleaning and Wrangling

['id','sex','age','age_cat','race','decile_score','score_text','priors_count', 
'c_charge_degree',  'is_recid','r_charge_degree','is_violent_recid',
'vr_charge_degree','v_decile_score', 'v_score_text',
'two_year_recid']

In [44]:
# Importing the raw dataset
raw_data = pd.read_csv("/Users/xu/Desktop/5243 project4/compas-scores-two-years.csv")
# raw_data.head()

# Create New Variable length_of_stay
length_of_stay = pd.to_datetime(raw_data["c_jail_out"]) - pd.to_datetime(raw_data["c_jail_in"])
length_of_stay_days = length_of_stay.astype('timedelta64[h]') / 24

# Filtering the features we need
filtered_df = raw_data.loc[
    (raw_data["race"].isin(["African-American", "Caucasian"])) &
    (length_of_stay_days > 0)
].assign(
    length_of_stay=length_of_stay_days
)[[
    "two_year_recid", "id", "sex", "race", "age", "age_cat", "decile_score", "score_text", "c_charge_degree", 
    "is_recid", "is_violent_recid", "v_decile_score", "v_score_text", "priors_count", "length_of_stay"
]]

# Drop missing values
filtered_df = filtered_df.dropna()

# Display filtered data
filtered_df.head(5)

,two_year_recid,id,sex,race,age,age_cat,decile_score,score_text,c_charge_degree,is_recid,is_violent_recid,v_decile_score,v_score_text,priors_count,length_of_stay
1,1,3,Male,African-American,34,25 - 45,3,Low,F,1,1,1,Low,0,10.041667
2,1,4,Male,African-American,24,Less than 25,4,Low,F,1,0,3,Low,4,1.083333
6,1,8,Male,Caucasian,41,25 - 45,6,Medium,F,1,0,2,Low,14,6.291667
8,0,10,Female,Caucasian,39,25 - 45,1,Low,M,0,0,1,Low,0,2.916667
9,1,13,Male,Caucasian,21,Less than 25,3,Low,F,1,1,5,Medium,1,0.958333


In [45]:
if (filtered_df['length_of_stay'] < 0).any():
    print("length_of_stay contains negative numbers")
else:
    print("length_of_stay does not contain negative numbers")

length_of_stay does not contain negative numbers


In [46]:
# Converting some of the features to be binary or categorical
filtered_df['sex'] = filtered_df['sex'].apply(lambda x: 1 if x == 'Male' else 0)
filtered_df['race'] = filtered_df['race'].apply(lambda x: 1 if x == 'Caucasian' else 0)

age_cat_map = {'Less than 25': 0, '25 - 45': 1, 'Greater than 45': 2}
filtered_df['age_cat'] = filtered_df['age_cat'].apply(lambda x: age_cat_map[x])

decile_score_map = {'Low': 0, 'Medium': 1, 'High': 2}
filtered_df['score_text'] = filtered_df['score_text'].apply(lambda x: decile_score_map[x])

filtered_df['c_charge_degree'] = filtered_df['c_charge_degree'].apply(lambda x: 1 if x == 'F' else 0)

v_score_map = {'Low': 0, 'Medium': 1, 'High': 2}
filtered_df['v_score_text'] = filtered_df['v_score_text'].apply(lambda x: v_score_map[x])

# filtered_df.head()

In [47]:
# Further features selection
cleaned_data = filtered_df.loc[:, ["two_year_recid", "sex", "race", "age_cat", "c_charge_degree", "v_score_text",
                                   "score_text", "priors_count", "length_of_stay"]]                       

# Normalize priors_count and length_of_stay
cleaned_data["priors_count"] = (cleaned_data["priors_count"] - cleaned_data["priors_count"].mean())/cleaned_data["priors_count"].std()
cleaned_data["length_of_stay"] = (cleaned_data["length_of_stay"] - cleaned_data["length_of_stay"].mean())/cleaned_data["length_of_stay"].std()

cleaned_data = cleaned_data.reset_index(drop=True)
cleaned_data.head()

# len(cleaned_data)       5699rows

,two_year_recid,sex,race,age_cat,c_charge_degree,v_score_text,score_text,priors_count,length_of_stay
0,1,1,0,1,1,0,0,-0.738411,-0.187151
1,1,1,0,0,1,0,0,0.045203,-0.356541
2,1,1,1,1,1,0,1,2.004240,-0.258059
3,0,0,1,1,0,0,0,-0.738411,-0.321875
4,1,1,1,0,1,1,0,-0.542508,-0.358905


        2.3 Splitting Dataframe

In [48]:
train_ratio = 0.7  # 70% for training
val_ratio = 0.15  # 15% for validation
test_ratio = 0.15  # 15% for testing

# Spliting the dataframe into train, val, and test with roughly 5:1:1
train_val, test = train_test_split(cleaned_data, test_size=test_ratio, random_state=110)
train, val = train_test_split(train_val, test_size=val_ratio/(train_ratio+val_ratio), random_state=110)

# print the shapes of the resulting dataframes
print(f"Training set shape: {train.shape}")
print(f"Validation set shape: {val.shape}")
print(f"Testing set shape: {test.shape}")

Training set shape: (3989, 9)
Validation set shape: (855, 9)
Testing set shape: (855, 9)


In [49]:
label = "two_year_recid"
sensitive_feature = "race"
features = list(cleaned_data.columns)

X_train = train.drop(label, axis=1)
Y_train = train[label].to_numpy()

X_val = val.drop(label, axis=1)
Y_val = val[label].to_numpy()

X_test = test.drop(label, axis=1)
Y_test = test[label].to_numpy()

race_train = train[sensitive_feature]
race_val = val[sensitive_feature]
race_test = test[sensitive_feature]

# 3. Functions defined

In [50]:
# Calibration
# Computes the calibration difference between Caucasians and African-Americans groups in the predicted outcomes.
def calibrate_difference(sensitive_features, y_pred, y_true):
    caucasians_index = np.where(sensitive_features == 1)[0]
    aa_index = np.where(sensitive_features == 0)[0]
    
    y_pred_caucasians = y_pred[caucasians_index]
    y_true_caucasians = y_true[caucasians_index]
    
    y_pred_aa = y_pred[aa_index]
    y_true_aa = y_true[aa_index]
    accuracy_caucasians = sum(y_pred_caucasians == y_true_caucasians) / len(y_true_caucasians)
    accuracy_aa = sum(y_pred_aa == y_true_aa) / len(y_true_aa)
    calib_diff = abs(accuracy_caucasians - accuracy_aa)
    return calib_diff


# Parity_ratio
def parity_ratio(y_pred, sensitive_features):
    assert len(y_pred) == len(sensitive_features), "Input arrays must have the same length"
    
    y_pred = np.array(y_pred)
    sensitive_features = np.array(sensitive_features)

    # Separate the data into two groups based on the sensitive feature
    y_pred_caucasians = y_pred[sensitive_features == 1]
    y_pred_aa = y_pred[sensitive_features == 0]

    # Calculate the probabilities of receiving a positive prediction for both groups
    prob_aa = np.mean(y_pred_aa == 1)
    prob_caucasians = np.mean(y_pred_caucasians == 1)

    # Calculate the parity ratio
    parity_ratio = prob_aa / prob_caucasians

    return parity_ratio



def p_rule(sensitive_features, y_pred):
    caucasians_index = np.where(sensitive_features == 1)[0]
    aa_index = np.where(sensitive_features == 0)[0]
    caucasians_pred = np.where(y_pred[caucasians_index] == 1)
    aa_pred = np.where(y_pred[aa_index] == 1)
    caucasians_percent = caucasians_pred[0].shape[0]/caucasians_index.shape[0]
    aa_percent = aa_pred[0].shape[0]/aa_index.shape[0]
    ratio = min(caucasians_percent/aa_percent, aa_percent/caucasians_percent)
    
    return ratio, caucasians_percent, aa_percent





# 4. Baseline

        3.1 Logistic Regression

In [51]:
# Fit a logistic regression classifier
lr_base = LogisticRegression(random_state=110)
lr_base.fit(X_train, Y_train)

# On validation set
accuracy_lr_base_val = lr_base.score(X_val, Y_val)
print(f"Accuracy on the validation set: {accuracy_lr_base_val}")

# Testing 
accuracy_lr_base_test = lr_base.score(X_test, Y_test)
print(f"Test accuracy: {accuracy_lr_base_test:.4f}")

Accuracy on the validation set: 0.672514619883041
Test accuracy: 0.6573


        3.2 SVM

In [52]:
# Fit an SVM classifier
svm_base = SVC(random_state=110)
svm_base.fit(X_train, Y_train)

# On validation
accuracy_svm_base_val = svm_base.score(X_val, Y_val)
print(f"Accuracy on the validation set: {accuracy_svm_base_val}")

# Testing
accuracy_svm_base_test = svm_base.score(X_test, Y_test)
print(f"Test accuracy: {accuracy_svm_base_test:.4f}")


Accuracy on the validation set: 0.6678362573099416
Test accuracy: 0.6585


        3.3 Summary of Baseline Models

In [53]:
summary_baseline = {"Methods": ["LR", "LR", "SVM", "SVM"], 
              "Set": ["Train", "Test", "Train", "Test"],
              "Accuracy (%)": [accuracy_lr_base_val*100, accuracy_lr_base_test*100, accuracy_svm_base_val*100, accuracy_svm_base_test*100],
              "Calibration(%)": [calibrate_difference(race_train, lr_base.predict(X_train), Y_train)*100,
                                 calibrate_difference(race_test, lr_base.predict(X_test), Y_test)*100,
                                 calibrate_difference(race_train, svm_base.predict(X_train), Y_train)*100,
                                 calibrate_difference(race_test, svm_base.predict(X_test), Y_test)*100],
              "parity_ratio": [p_rule(race_train, lr_base.predict(X_train))[0],
                                 p_rule(race_test, lr_base.predict(X_test))[0],
                                 p_rule(race_train, svm_base.predict(X_train))[0],
                                 p_rule(race_test, svm_base.predict(X_test))[0]]}
pd.DataFrame(summary_baseline)

,Methods,Set,Accuracy (%),Calibration(%),parity_ratio
0,LR,Train,67.251462,2.555860,0.495499
1,LR,Test,65.730994,0.393226,0.544211
2,SVM,Train,66.783626,3.488891,0.579274
3,SVM,Test,65.847953,2.654994,0.634721


# 5. Optimization with Fairness Constraints (Paper A2)

        4.1 CLR

In [54]:
x_control = {'race': race_train}
loss_function = lf._logistic_loss

apply_fairness_constraints = 1 # set this flag to 1 since we want to optimize accuracy subject to fairness constraints
apply_accuracy_constraint = 0

sep_constraint = 0
gamma = None
sensitive_attrs = ['race']
sensitive_attrs_to_cov_thresh = {'race': 0}


# Train model
np.random.seed(100)
w = ut.train_model(X_train,
                   Y_train,
                   x_control,
                   loss_function,
                   apply_fairness_constraints,
                   apply_accuracy_constraint,
                   sep_constraint,
                   sensitive_attrs,
                   sensitive_attrs_to_cov_thresh,
                   gamma)


# Fit coefficients/weights into logistic regression in sklearn
CLR = LogisticRegression()
CLR.coef_= w.reshape((1,-1))
CLR.intercept_ = 0
CLR.classes_ = np.array([0, 1])

In [55]:


# On validation set
accuracy_CLR_val = CLR.score(X_val, Y_val)
print(f"Accuracy on the validation set: {accuracy_CLR_val}")

# Testing 
accuracy_CLR_test = CLR.score(X_test, Y_test)
print(f"Test accuracy: {accuracy_CLR_test:.4f}")

Accuracy on the validation set: 0.48771929824561405
Test accuracy: 0.4982


/Users/xu/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/xu/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


        4.2 CSVM

In [56]:
# result_CLR = {"Classifier": ["C-LR", "C-LR"],
#                "Set": ["Train", "Test"],
#                "P-rule (%)": [p_rule(race_train, m.predict(X_train))[0]*100, p_rule(race_test, m.predict(X_test))[0]*100],
#                "Accuracy (%)": [m.score(X_train, Y_train)*100, m.score(X_test, Y_test)*100], 
#                "Protected (%)": [p_rule(race_train, m.predict(X_train))[1]*100, p_rule(race_test, m.predict(X_test))[1]*100],
#                "Not protected (%)": [p_rule(race_train, m.predict(X_train))[2]*100, p_rule(race_test, m.predict(X_test))[2]*100],
#                "Calibration (%)": [calibrate_difference(race_train, m.predict(X_train), Y_train)*100, calibrate_difference(race_test, m.predict(X_test), Y_test)*100]
#              }
# pd.DataFrame(result_CLR)

In [57]:
summary_A2 = {"Methods": ["CLR", "CLR", "CSVM", "SVM"], 
              "Set": ["Train", "Test", "Train", "Test"],
              "Accuracy (%)": [accuracy_CLR_val*100, accuracy_CLR_test*100, accuracy_svm_base_val*100, accuracy_svm_base_test*100],
              "Calibration(%)": [calibrate_difference(race_train, CLR.predict(X_train), Y_train)*100,
                                 calibrate_difference(race_test, CLR.predict(X_test), Y_test)*100,
                                 calibrate_difference(race_train, svm_base.predict(X_train), Y_train)*100,
                                 calibrate_difference(race_test, svm_base.predict(X_test), Y_test)*100],
              "parity_ratio": [p_rule(race_train, CLR.predict(X_train))[0],
                                 p_rule(race_test, CLR.predict(X_test))[0],
                                 p_rule(race_train, svm_base.predict(X_train))[0],
                                 p_rule(race_test, svm_base.predict(X_test))[0]]}

pd.DataFrame(summary_A2)

/Users/xu/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/xu/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/xu/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/xu/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


,Methods,Set,Accuracy (%),Calibration(%),parity_ratio
0,CLR,Train,48.771930,13.514380,0.999584
1,CLR,Test,49.824561,8.791619,0.998936
2,CSVM,Train,66.783626,3.488891,0.579274
3,SVM,Test,65.847953,2.654994,0.634721


# 6. Local Massaging & Local Preferential Sampling (Paper A6)

        5.1 Local Massaging

In [58]:

#df is the dataframe with all columns to be partitioned
#e is the name of explainatory varaible
def PARTITION(df, e):
    groups = []
    uniques = np.unique(df[e])
    for u in uniques:
        groups.append(df[df[e]==u])
    return groups
    
#part is the dataframe with all columns with same value for the explainatory variable
#si is the current sensitive parameter value
def DELTA(df, part, si):
    isRace = part['race']==si
    Gi = sum(isRace)
    n = len(df)
    
    num = sum(part[isRace]['two_year_recid']==1)/n
    denom = len(part[isRace])/n
    P = num/denom
    
    isNotRace = part['race']!=si
    num = sum(part[isNotRace]['two_year_recid']==1)/n
    denom = len(part[isNotRace])/n

    P_star = 0.5*(P + num/denom) #explainable difference
    return np.floor(Gi*abs(P-P_star)).astype(np.int64)

In [59]:
#Local Massaging algorithm
LM_labels = []
for part in PARTITION(train, 'c_charge_degree'):
    #train model
    X_part = part.drop('two_year_recid', axis=1)
    y_part = part['two_year_recid']
    log_model = LogisticRegression(random_state=110)
    model=log_model.fit(X_part, y_part)
    
    #predict labels from model
    part1 = part[part['race']==1]
    part1.reset_index(drop=True, inplace=True)
    delta1 = DELTA(train, part, 1)
    X_part1 = part1.drop('two_year_recid', axis=1)
    y_part1 = part1['two_year_recid']
    rank = pd.DataFrame(model.decision_function(X_part1), columns = ['rank'])
    comb1 = pd.concat([part1, rank], axis=1)
    
    part0 = part[part['race']==0]
    part0.reset_index(drop=True, inplace=True)
    delta0 = DELTA(train, part, 0)
    X_part0 = part0.drop('two_year_recid', axis=1)
    y_part0 = part0['two_year_recid']
    rank = pd.DataFrame(model.decision_function(X_part0), columns = ['rank'])
    comb0 = pd.concat([part0, rank], axis=1)

    #for C, relabel closest delta datapoints from C to AA
    comb1 = comb1.sort_values(['rank'])
    comb1.reset_index(drop=True, inplace=True)
    
    t = sum(comb1['rank']>0)
    l = len(comb1)
    
    fix1 = np.full(l-t, False)
    relabel = np.full(delta1, True)
    fix2 = np.full(t-delta1, False)
    comb1.loc[np.concatenate([fix1, relabel, fix2]), 'two_year_recid'] = 0
    LM_labels.append(comb1)
    
    #for AA, relabel closest delta datapoints from AA to C
    comb0 = comb0.sort_values(['rank'])
    comb0.reset_index(drop=True, inplace=True)
    
    t = sum(comb0['rank']<0)
    l = len(comb0)
    
    fix1 = np.full(t-delta0, False)
    relabel = np.full(delta0, True)
    fix2 = np.full(l-t, False)
    comb0.loc[np.concatenate([fix1, relabel, fix2]), 'two_year_recid'] = 1
    LM_labels.append(comb0)
    
loc_mass = pd.concat(LM_parts, axis=0)

In [60]:
features=["sex", "race", "age_cat", "c_charge_degree", "v_score_text",
                                   "score_text", "priors_count", "length_of_stay"]
X_train = loc_mass[features]
y_train = loc_mass['two_year_recid']
log_model = LogisticRegression(random_state=110)
classifier=log_model.fit(X_train, y_train)
classifier.score(X_val, Y_val)

0.6736842105263158

        5.2 Local Preferential Sampling


In [61]:
#Local Preferential Sampling
LPS_labels = []
partition = PARTITION(train, 'c_charge_degree')
for part in partition:
    #train model
    X_part = part.drop("two_year_recid", axis=1)
    y_part = part["two_year_recid"]
    log_model = LogisticRegression(random_state=110)
    model = log_model.fit(X_part, y_part)
    
    #predict labels from model
    part1 = part[part["race"]==1]
    part1.reset_index(drop=True, inplace=True)
    delta1 = DELTA(train, part, 1)//2
    X_part1 = part1.drop("two_year_recid", axis=1)
    y_part1 = part1["two_year_recid"]
    rank = pd.DataFrame(model.decision_function(X_part1), columns = ['rank'])
    comb1 = pd.concat([part1, rank], axis=1)
    
    part0 = part[part["race"]==0]
    part0.reset_index(drop=True, inplace=True)
    delta0 = DELTA(train, part, 0)//2
    X_part0 = part0.drop("two_year_recid", axis=1)
    y_part0 = part0["two_year_recid"]
    rank = pd.DataFrame(model.decision_function(X_part0), columns = ['rank'])
    comb0 = pd.concat([part0, rank], axis=1)

    #for C, replace closest delta/2 data with duplicates from same number of AA 
    comb1 = comb1.sort_values(["rank"])
    comb1.reset_index(drop=True, inplace=True)
    
    t = sum(comb1["rank"]>0)
    l = len(comb1)
    
    keep1 = np.full(l-t, False)
    replace = np.full(delta1, True)
    keep2 = np.full(t-delta1, False)
    toKeep = np.invert(np.concatenate([keep1, replace, keep2]))
    dup1 = np.full(l-t-delta1, False)
    dup2 = np.full(t, False)
    toDup = np.concatenate([dup1, replace, dup2])
    duplicates = comb1[toDup]
    comb1 = comb1[toKeep]
    comb1 = pd.concat([comb1,duplicates], axis=0)
    LPS_labels.append(comb1)
    
    #for AA, replace closest delta/2 data with duplicates from same number of C 
    comb0 = comb0.sort_values(['rank'])
    comb0.reset_index(drop=True, inplace=True)
    
    t = sum(comb0['rank']<0)
    l = len(comb0)
    
    keep1 = np.full(t-delta0, False)
    replace = np.full(delta0, True)
    keep2 = np.full(l-t, False)
    toKeep = np.invert(np.concatenate([keep1, replace, keep2]))
    dup1 = np.full(t, False)
    dup2 = np.full(l-t-delta0, False)
    toDup = np.concatenate([dup1, replace, dup2])
    duplicates = comb0[toDup]
    comb0 = comb0[toKeep]
    comb0 = pd.concat([comb0,duplicates], axis=0)
    LPS_labels.append(comb0)